## α-β-Pruning

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game.ipynb
%run ./nmm-artificial-intelligence.ipynb
%run ./nmm-heuristic.ipynb
%run ./nmm-symmetry.ipynb
import time

In [ ]:
class AlphaBetaPruning(ArtificialIntelligence):
    def __init__(self, max_states=25_000, symmetry=True, weights=HeuristicWeights()):
        self.cache = {}
        self.oldCache = {}
        
        self.max_states = max_states
        self.symmetry = symmetry
        self.weights = weights
    
    def bestMoves(self, state, player):
        pass

Die Hilfsfunktion `writeCache_AB` legt den gegebenen Zustand `state` und Spieler `player` mit samt der errechneten Werte `value`, `alpha` und `beta` im Cache ab. Ist zusätzlich der Parameter `symmetry` auf `True` gesetzt, werden die errechneten Werte ebenfalls für alle symmetrischen Zustände gespeichert.

In [ ]:
def writeCache(self, state, player, value, alpha, beta):
    # If symmetry is enabled, find all symmetic states and save them too
    if self.symmetry:
        for symmetricState in findSymmetries(state):
            self.cache[(symmetricState, player)] = (value, alpha, beta)
    else:
        self.cache[(state, player)] = (value, alpha, beta)

AlphaBetaPruning.writeCache = writeCache
del writeCache

Die Exception `IterativeMaxCountException` wird aufgerufen, wenn ein Fehler aufgrund des Erreichens der maximal zu betrachtenden Zustände bei der iterativen Tiefensuche erzeugt werden soll.

In [ ]:
class IterativeMaxCountException(Exception):
    def __init__(self, maxCount):
        super().__init__(f"Reached max count ({maxCount}) of iterative deepeing")    

Die Klasse `IterativeCounter` implementiert zum einen die Werte `maxCount` und `value` und zum anderen die Methoden `increment` und `check`.
Die Methode `increment` erhöht den `value` im eins, in der Methoden `check` wird zusätzlich noch überprüft, ob der neu erhöhte `value` den `maxCount` überschreitet und erzeugt eine `IterativeMaxCountException`.

Die Implementierung als Klasse ermöglicht es, eine Referenz eines Objekts der Klasse an die Funktionen `value_AB` und `alphaBeta` zu übergeben. Dadurch wird effektiv in allen Teilbäumen und allen Rekursionstiefen das gleiche Objekt aufgerufen und somit der `value` korrekt gezählt.

In [ ]:
def checkMaxStates(self):
    self.visited += 1
    if (self.visited >= self.max_states):
        raise IterativeMaxCountException(self.max_states)

AlphaBetaPruning.checkMaxStates = checkMaxStates
del checkMaxStates

Die Funktion `orderMoves` sortiert eine Liste von nächsten Zuständen `ns` für einen Spieler `p` entsprechend der für diese Zustände erreichten Werte aus der vorherigen Iteration `OldCache`. So soll erreicht werden, dass vielversprechende Zustände früher betrachtet werden sollen und sich somit Teilbäume eventuell früher abschneiden lassen.

In [ ]:
def orderMoves(self, states, player):
    return sorted(states, key = lambda state: self.oldCache.get((state, player), (0, -1, 1))[0], reverse=True)

AlphaBetaPruning.orderMoves = orderMoves
del orderMoves

Die `value()` Funktion ist ein Wrapper für die eigentliche Implementierung des α-β-Pruning in der Funktion `alphaBeta()`. Dieser Wrapper stellt Caching der Ergebnisse zur Verfügung, bzw. berechnet diese neu, falls der Cache keine oder invalide Ergenisse beinhaltet.

Ein Ergebnis aus dem Cache ist valide, solange das Intervall `alpha` und `beta` aus den Parametern innerhalb des im Cache verwendeten Intervalls `a` und `b` liegt. Also das Intervall des Caches muss genereller sein, als das Intervall aus den Paramtern.

In [ ]:
def value(self, state, player, alpha, beta, limit):
    if (state, player) in self.cache:
        (val, a, b) = self.cache[(state, player)]
        if a <= alpha and beta <= b:
            self.cache_hit += 1
            return val
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            val   = self.alphaBeta(state, player, alpha, beta, limit)
            self.writeCache(state, player, val, alpha, beta)
            self.cache_miss += 1
            return val
    else:
        val = self.alphaBeta(state, player, alpha, beta, limit)
        self.writeCache(state, player, val, alpha, beta)
        self.cache_miss += 1
        return val

AlphaBetaPruning.value = value
del value

Die Funktion `alphaBeta()` beinhaltet nun die eigentliche Implementierung des α-β-Pruning.

* Wie zuvor beim Minimax Algorithmus, wird der `utility` Wert zurückgegeben, falls das Spiel in dem State `s` beendet (`finished`) ist.
* Ebenfalls äquivalent wird der `heuristic` Wert verwendet, sobald das Rekursionslimit (`limit`) erreicht wird.
* Außerdem wird der Zähler für die iterative Tiefensuche erhöht, sofern ein Objekt der Klasse `IterativeCounter` übergeben wird.
* Der eigentliche α-β-Pruning Alogrithmus errechnet rekursiv mit Hilfe des Caches (`value_AB()`) den Wert eines Zuges. Hierbei wird der erste Wert der nächsten States verwendet, der größer oder gleich der oberen Grenze `beta` ist.

In [ ]:
def alphaBeta(self, state, player, alpha, beta, limit):
    self.checkMaxStates()
    
    if limit == 0:
        return heuristic(state, player)

    states = nextStates(state, player)
    if finished(state, player, ns=states):
        return utility(state, player, ns=states)

    val = alpha
    for ns in self.orderMoves(states, player):
        val = max(val, -self.value(ns, opponent(player), -beta, -alpha, limit-1))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

AlphaBetaPruning.alphaBeta = alphaBeta
del alphaBeta

`bestMoves()` wählt mit Hilfe des α-β-Pruning Algorithmus den besten State aus allen möglichen nächsten States aus. Hierzu werden die Werte aller States errechnet und der State mit dem höchsten Wert ausgewählt. Dies wird solange ausgeführt, bis der maximale Wert der iterativen Tiefensuche erreicht ist und einen Fehler erzeugt.

In [ ]:
def bestMoves(self, state, player):
    # Start clock
    start = time.time()
    
    # Reset counter
    self.visited = 0
    self.cache_hit = 0
    self.cache_miss = 0
    
    # Clear cache
    self.cache = {}
    self.oldCache = {}
    
    ns = nextStates(state, player)
    moves = [(0, s) for s in ns]

    limit = 1
    while True:
        try:
            moves = [
                (-self.value(state, opponent(player), -1, 1, limit), state)
                for s in ns
            ]
            self.oldCache = self.cache
            self.cache = {}
            limit += 1
        except IterativeMaxCountException:
            break
    
    maximum = max(value for (value, state) in moves)
    bestMoves = [state for (value, state) in moves if value == maximum]
    
    end = time.time()
    return BestMoves(
        bestMoves,
        maximum,
        # Collect debug information
        {   "runtime": end - start,
            "limit": limit,
            "visited": self.visited,
            "max_states": self.max_states,
            "cache_hit": self.cache_hit,
            "cache_miss": self.cache_miss,   }
    )

AlphaBetaPruning.bestMoves = bestMoves
del bestMoves